In [ ]:
# Install TabPFN
!git clone https://github.com/PriorLabs/tabpfn
!pip install -e tabpfn

# TabPFN Community installs optional functionalities around the TabPFN model
# These include post-hoc ensembles, interpretability tools, and more
!git clone https://github.com/PriorLabs/tabpfn-extensions
!pip install -e tabpfn-extensions[post_hoc_ensembles,interpretability,hpo]

# Install TabICL
!pip install git+https://github.com/soda-inria/tabicl.git

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from tqdm.notebook import tqdm

from sklearn.metrics import roc_auc_score, brier_score_loss
from sklearn.calibration import calibration_curve
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Deep learning methods
from tabpfn_extensions.post_hoc_ensembles.sklearn_interface import AutoTabPFNClassifier
from tabpfn_extensions.rf_pfn import (
    RandomForestTabPFNClassifier,
)

from tabpfn_extensions import TabPFNClassifier

from tabicl import TabICLClassifier

from joblib import Parallel, delayed
import pickle
import timeit

import torch
if not torch.cuda.is_available():
    raise SystemError('GPU device not found. For fast training, please enable GPU. See section above for instructions.')

In [ ]:
mimic_3 = pd.read_csv('mimic_3_processed.csv')
mimic_4 = pd.read_csv('mimic_4_processed.csv')

In [ ]:
features = [
    'age', 'heartrate_max', 'heartrate_min', 'sysbp_max', 'sysbp_min', 'tempc_max', 'tempc_min', 
    'urineoutput', 'bun_min', 'bun_max', 'wbc_min', 'wbc_max',
    'potassium_min', 'potassium_max', 'sodium_min', 'sodium_max', 'bicarbonate_min', 'bicarbonate_max', 
    'mingcs', 'aids', 'hem', 'mets', 'admissiontype',
#    'pao2fio2_vent_min', 'bilirubin_min', 'bilirubin_max',
]

cont_features = [
    'age', 'heartrate_max', 'heartrate_min', 'sysbp_max', 'sysbp_min', 'tempc_max', 'tempc_min', 
    'urineoutput', 'bun_min', 'bun_max', 'wbc_min', 'wbc_max',
    'potassium_min', 'potassium_max', 'sodium_min', 'sodium_max', 'bicarbonate_min', 'bicarbonate_max', 
    'mingcs'
#    'pao2fio2_vent_min', 'bilirubin_min', 'bilirubin_max',
]

cat_features = [
    'aids', 'hem', 'mets', 'admissiontype'
]

outcome = [
    'hospital_mortality'
]

socio_demographic = [
    'insurance', 'marital_status', 'ethnicity', 'language', 'gender'
]

In [ ]:
X = mimic_3[features]
y = mimic_3['hospital_mortality']
y_proxy = mimic_3['icustay_expire_flag']

In [ ]:
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

In [ ]:
def process_df(df, n_folds = n_folds):
    df_true_prob = df['Prob true'].explode().groupby(level=0).apply(lambda x: pd.Series(x.values)).unstack()
    df_true_prob.columns = ['Prob_true_fold_' + str(i+1) for i in range(n_folds)]
    df_pred_prob = df['Prob pred'].explode().groupby(level=0).apply(lambda x: pd.Series(x.values)).unstack()
    df_pred_prob.columns = ['Prob_pred_fold_' + str(i+1) for i in range(n_folds)]
    df = df.drop(['Prob true', 'Prob pred'], axis=1)
    df = pd.concat([df, df_true_prob, df_pred_prob], axis=1)
    return df

In [ ]:
def create_directory(directory_name):
    try:
        os.mkdir(directory_name)
        print(f"Directory '{directory_name}' created successfully.")
    except FileExistsError:
        print(f"Directory '{directory_name}' already exists.")

In [ ]:
create_directory('results')

In [ ]:
clf_base = TabPFNClassifier(
    ignore_pretraining_limits=True,
    inference_config = {"SUBSAMPLE_SAMPLES": 10000} # Needs to be set low so that not OOM on fitting intermediate nodes
)

tabpfn_tree_clf = RandomForestTabPFNClassifier(
    tabpfn=clf_base,
    verbose=1,
    max_predict_time=60, # Will fit for one minute
    fit_nodes=True, # Wheather or not to fit intermediate nodes
    adaptive_tree=True, # Whather or not to validate if adding a leaf helps or not
  )

# Define models
models = {
    'TabPFN RF': tabpfn_tree_clf,
    'TabICL': TabICLClassifier(),
}

In [ ]:
models_name = list(models.keys())

n_models = len(models_name)

In [ ]:
score_crossvalidation = Parallel(n_jobs=-1, verbose=1)(delayed(cross_val_score)(model, X, y, scoring='roc_auc', cv=n_folds) 
                                          for model in models.values()
                                         )

In [ ]:
score_crossvalidation_df = pd.DataFrame(
    score_crossvalidation, index=models_name, columns=["fold_"+str(i+1) for i in np.arange(n_folds)]
)

In [ ]:
score_crossvalidation_df.to_csv('results/result_crossvalidation.csv')

## Robustness tests

### Label noise

In [ ]:
random_label_noise_levels = np.linspace(0, 1, 11)
targeted_label_noise_levels = np.linspace(0, 1, 10, endpoint=False)

In [ ]:
def add_label_noise(y_noisy, noise_level):
    idx_change_outcome = np.random.rand(len(y_noisy))<noise_level
    y_noisy[idx_change_outcome] = 1-y_noisy
    
    return y_noisy

In [ ]:
def label_noise(model_name, model, noise_level, train_idx, val_idx, noise_type='random'):

    X_train, X_val = X.loc[train_idx], X.loc[val_idx]
    y_train, y_val = y.loc[train_idx], y.loc[val_idx]
    y_proxy_train = y_proxy.loc[train_idx]
    y_train_noisy = y_train.copy()
    
    if noise_type=='random':
        y_train_noisy = add_label_noise(y_train_noisy, noise_level)
    elif noise_type=='0to1':
        y_train_noisy[y_train_noisy==0] = add_label_noise(y_train_noisy[y_train_noisy==0], noise_level)
    elif noise_type=='1to0':
        y_train_noisy[y_train_noisy==1] = add_label_noise(y_train_noisy[y_train_noisy==1], noise_level)
    elif noise_type=='conditional':
        age_train_perc = X_train.age.rank(pct=True)
        swap_by_age = np.random.binomial(1, p=age_train_perc**noise_level, size=len(age_train_perc))
        y_train_noisy = y_train_noisy*(swap_by_age) + (1-y_train_noisy)*(1-swap_by_age)
    elif noise_type=='proxy':
        y_train_noisy = y_train_noisy*(1-noise_level) + y_proxy_train*noise_level
        
    
    model.fit(X_train, y_train_noisy)

    y_pred = model.predict_proba(X_val)[:,1]
    prob_true, prob_pred = calibration_curve(y_val, y_pred)
    
    return model_name, noise_level, roc_auc_score(y_score=y_pred, y_true=y_val), brier_score_loss(y_true=y_val, y_proba=y_pred), prob_true, prob_pred

In [ ]:
results_random_label_noise = Parallel(n_jobs=-1, verbose=1)(delayed(label_noise)(model_name, model, noise_level, train_idx, val_idx, noise_type='random') 
                                          for model_name, model in models.items()
                                          for noise_level in random_label_noise_levels
                                          for train_idx, val_idx in kf.split(X)
                                         )

In [ ]:
results_label_noise_01 = Parallel(n_jobs=-1, verbose=1)(delayed(label_noise)(model_name, model, noise_level, train_idx, val_idx, noise_type='0to1') 
                                          for model_name, model in models.items()
                                          for noise_level in targeted_label_noise_levels
                                          for train_idx, val_idx in kf.split(X)
                                         )

In [ ]:
results_label_noise_10 = Parallel(n_jobs=-1, verbose=1)(delayed(label_noise)(model_name, model, noise_level, train_idx, val_idx, noise_type='1to0') 
                                          for model_name, model in models.items()
                                          for noise_level in targeted_label_noise_levels
                                          for train_idx, val_idx in kf.split(X)
                                         )

In [ ]:
results_label_noise_age = Parallel(n_jobs=-1, verbose=1)(delayed(label_noise)(model_name, model, noise_level, train_idx, val_idx, noise_type='conditional') 
                                          for model_name, model in models.items()
                                          for noise_level in random_label_noise_levels
                                          for train_idx, val_idx in kf.split(X)
                                         )

In [ ]:
results_label_noise_proxy = Parallel(n_jobs=-1, verbose=1)(delayed(label_noise)(model_name, model, noise_level, train_idx, val_idx, noise_type='proxy') 
                                          for model_name, model in models.items()
                                          for noise_level in [0, 1]
                                          for train_idx, val_idx in kf.split(X)
                                         )

In [ ]:
directory = 'results/label_noise/'
create_directory(directory)

label_noise_levels = {
    'random': random_label_noise_levels,
    '0to1': targeted_label_noise_levels,
    '1to0': targeted_label_noise_levels,
    'age': random_label_noise_levels,
    'proxy': [0,1]
}

results_label_noise = {
    "random": results_random_label_noise, 
    "0to1": results_label_noise_01, 
    "1to0": results_label_noise_10,
    "age": results_label_noise_age, 
    "proxy": results_label_noise_proxy
}

for k, result in results_label_noise.items():
    df_results = pd.DataFrame(result, columns=['Model', 'Noise level', 'AUC', 'Brier score', 'Prob true', 'Prob pred'])
    df_results.to_csv(directory+k+'.csv', index=False)
    df_results = process_df(df_results, n_folds)    
    df_results.to_csv(directory+k+'.csv', index=False)

### Measurement noise
---

In [ ]:
input_noise_level = np.linspace(0,1,11)

In [ ]:
def add_measurement_noise(X, noise_level, feature_type='cont & cat'):

    X_noisy = X.copy()
    size = X_noisy.shape
    
    if 'cont' in feature_type:
        for j in cont_features:
            std_j = X_noisy[j].std()
            noise_j = np.random.normal(scale=std_j*noise_level, size=size[0])
            X_noisy[j] = X_noisy[j] + noise_j
            
    if 'cat' in feature_type:
        for j in cat_features:
            max_xj = X_noisy[j].max()
            min_xj = X_noisy[j].min()
            mask = np.random.binomial(1, noise_level, size=size[0])
            noise = np.random.randint(0, max_xj, size=size[0]) ## only works for consecutive integers
            noise[noise == X_noisy[j].values] = (noise[noise == X_noisy[j].values] + 1) % max_xj
            X_noisy[j] = np.where(mask, noise, X_noisy[j])
    
    return X_noisy

In [ ]:
def input_noise(model_name, model, noise_level, train_idx, val_idx, which_set="Train", feature_type='cont & cat'):

    X_train, X_val = X.loc[train_idx], X.loc[val_idx]
    y_train, y_val = y.loc[train_idx], y.loc[val_idx]
    
    if which_set=="Train":
        X_train = add_measurement_noise(X_train, noise_level, feature_type)
    if which_set=="Val":
        X_val = add_measurement_noise(X_val, noise_level, feature_type)
    if which_set=="Train_Val":
        X_train = add_measurement_noise(X_train, noise_level, feature_type)
        X_val = add_measurement_noise(X_val, noise_level, feature_type)

    model.fit(X_train, y_train)

    y_pred = model.predict_proba(X_val)[:,1]
    prob_true, prob_pred = calibration_curve(y_val, y_pred)
    
    return model_name, noise_level, roc_auc_score(y_score=y_pred, y_true=y_val), brier_score_loss(y_true=y_val, y_proba=y_pred), prob_true, prob_pred

In [ ]:
results_input_noise_train = Parallel(n_jobs=-1, verbose=1)(delayed(input_noise)(model_name, model, noise_level, train_idx, val_idx) 
                                          for model_name, model in models.items()
                                          for noise_level in input_noise_level
                                          for train_idx, val_idx in kf.split(X)
                                         )

In [ ]:
results_input_noise_val = Parallel(n_jobs=-1, verbose=1)(delayed(input_noise)(model_name, model, noise_level, train_idx, val_idx, which_set='Val') 
                                          for model_name, model in models.items()
                                          for noise_level in input_noise_level
                                          for train_idx, val_idx in kf.split(X)
                                         )

In [ ]:
results_input_noise_all = Parallel(n_jobs=-1, verbose=1)(delayed(input_noise)(model_name, model, noise_level, train_idx, val_idx, which_set='Train_Val') 
                                          for model_name, model in models.items()
                                          for noise_level in input_noise_level
                                          for train_idx, val_idx in kf.split(X)
                                         )

In [ ]:
results_input_noise_train_cont = Parallel(n_jobs=-1, verbose=1)(delayed(input_noise)(model_name, model, noise_level, train_idx, val_idx, feature_type='cont') 
                                          for model_name, model in models.items()
                                          for noise_level in input_noise_level
                                          for train_idx, val_idx in kf.split(X)
                                         )
results_input_noise_val_cont = Parallel(n_jobs=-1, verbose=1)(delayed(input_noise)(model_name, model, noise_level, train_idx, val_idx, which_set='Val', feature_type='cont') 
                                          for model_name, model in models.items()
                                          for noise_level in input_noise_level
                                          for train_idx, val_idx in kf.split(X)
                                         )
results_input_noise_all_cont = Parallel(n_jobs=-1, verbose=1)(delayed(input_noise)(model_name, model, noise_level, train_idx, val_idx, which_set='Train_Val', feature_type='cont') 
                                          for model_name, model in models.items()
                                          for noise_level in input_noise_level
                                          for train_idx, val_idx in kf.split(X)
                                         )

In [ ]:
results_input_noise_train_cat = Parallel(n_jobs=-1, verbose=1)(delayed(input_noise)(model_name, model, noise_level, train_idx, val_idx, feature_type='cat') 
                                          for model_name, model in models.items()
                                          for noise_level in input_noise_level
                                          for train_idx, val_idx in kf.split(X)
                                         )
results_input_noise_val_cat = Parallel(n_jobs=-1, verbose=1)(delayed(input_noise)(model_name, model, noise_level, train_idx, val_idx, which_set='Val', feature_type='cat') 
                                          for model_name, model in models.items()
                                          for noise_level in input_noise_level
                                          for train_idx, val_idx in kf.split(X)
                                         )
results_input_noise_all_cat = Parallel(n_jobs=-1, verbose=1)(delayed(input_noise)(model_name, model, noise_level, train_idx, val_idx, which_set='Train_Val', feature_type='cat') 
                                          for model_name, model in models.items()
                                          for noise_level in input_noise_level
                                          for train_idx, val_idx in kf.split(X)
                                         )

In [ ]:
directory = 'results/input_noise/'
create_directory(directory)

results_input_noise = {
    "train": results_input_noise_train, 
    "val": results_input_noise_val, 
    "all": results_input_noise_all,
    "train_cont": results_input_noise_train_cont, 
    "val_cont": results_input_noise_val_cont, 
    "all_cont": results_input_noise_all_cont, 
    "train_cat": results_input_noise_train_cat, 
    "val_cat": results_input_noise_val_cat, 
    "all_cat": results_input_noise_all_cat, 
}

for k,result in results_input_noise.items():
    df_results = pd.DataFrame(result, columns=['Model', 'Noise level', 'AUC', 'Brier score', 'Prob true', 'Prob pred'])
    df_results.to_csv(directory+k+'.csv', index=False)
    df_results = process_df(df_results, n_folds)    
    df_results.to_csv(directory+k+'.csv', index=False)

### Imbalance data

---


In [ ]:
imbalance_ratio = np.linspace(1, 0, 10, endpoint=False)

In [ ]:
def imbalance_data(model_name, model, imbalance_ratio, train_idx, val_idx):

    X_train, X_val = X.loc[train_idx], X.loc[val_idx]
    y_train, y_val = y.loc[train_idx], y.loc[val_idx]

    X_train_negative = X_train.loc[y_train==0].sample(frac = imbalance_ratio)
    y_train_negative = y_train.loc[X_train_negative.index]
    X_train_balanced = pd.concat([X_train_negative, X_train.loc[y_train==1]])
    y_train_balanced = pd.concat([y_train_negative, y_train.loc[y_train==1]])

    model.fit(X_train_balanced, y_train_balanced)

    y_pred = model.predict_proba(X_val)[:,1]
    prob_true, prob_pred = calibration_curve(y_val, y_pred)
    
    return model_name, imbalance_ratio, roc_auc_score(y_score=y_pred, y_true=y_val), brier_score_loss(y_true=y_val, y_proba=y_pred), prob_true, prob_pred

In [ ]:
results_imbalance_data = Parallel(n_jobs=-1, verbose=1)(delayed(imbalance_data)(model_name, model, ratio, train_idx, val_idx) 
                                          for model_name, model in models.items()
                                          for ratio in imbalance_ratio
                                          for train_idx, val_idx in kf.split(X)
                                         )

In [ ]:
directory = 'results/imbalance_data/'
create_directory(directory)

df_results = pd.DataFrame(results_imbalance_data, columns=['Model', 'Noise level', 'Brier score', 'AUC', 'Prob true', 'Prob pred'])
df_results.to_csv(directory+'imbalance_data.csv', index=False)
df_results = process_df(df_results, n_folds)    
df_results.to_csv(directory+'imbalance_data.csv', index=False)

### Feature shuffling

In [ ]:
shuffle_ratio = np.linspace(0, 1, 11, endpoint=True)

In [ ]:
def shuffle_features(X, prop=0.5, feat_to_shuffle=None):
    X_noisy = X.copy()
    size = X_noisy.shape

    n_feat_to_shuffle = int(size[1]*prop)
    if feat_to_shuffle is None:
        feat_to_shuffle = np.random.choice(X_noisy.columns, size=n_feat_to_shuffle, replace=False)
    X_noisy[feat_to_shuffle] = X_noisy[feat_to_shuffle].apply(np.random.permutation)
    
    return X_noisy, feat_to_shuffle

In [ ]:
def permutation_features(model_name, model, noise_level, train_idx, val_idx, which_set="Train"):

    X_train, X_val = X.loc[train_idx], X.loc[val_idx]
    y_train, y_val = y.loc[train_idx], y.loc[val_idx]
    
    if which_set=="Train":
        X_train, _ = shuffle_features(X_train, noise_level)
    if which_set=="Val":
        X_val, _ = shuffle_features(X_val, noise_level)
    if which_set=="Train_Val":
        X_train, feat_to_shuffle = shuffle_features(X_train, noise_level)
        X_val, _ = shuffle_features(X_val, noise_level, feat_to_shuffle=feat_to_shuffle)

    model.fit(X_train, y_train)

    y_pred = model.predict_proba(X_val)[:,1]
    prob_true, prob_pred = calibration_curve(y_val, y_pred)
    
    return model_name, noise_level, roc_auc_score(y_score=y_pred, y_true=y_val), brier_score_loss(y_true=y_val, y_proba=y_pred), prob_true, prob_pred    

In [ ]:
results_shuffled_data_train = Parallel(n_jobs=-1, verbose=1)(delayed(permutation_features)(model_name, model, ratio, train_idx, val_idx, which_set='Train') 
                                          for model_name, model in models.items()
                                          for ratio in shuffle_ratio
                                          for train_idx, val_idx in kf.split(X)
                                         )
results_shuffled_data_val = Parallel(n_jobs=-1, verbose=1)(delayed(permutation_features)(model_name, model, ratio, train_idx, val_idx, which_set='Val') 
                                          for model_name, model in models.items()
                                          for ratio in shuffle_ratio
                                          for train_idx, val_idx in kf.split(X)
                                         )
results_shuffled_data_all = Parallel(n_jobs=-1, verbose=1)(delayed(permutation_features)(model_name, model, ratio, train_idx, val_idx, which_set='Train_Val') 
                                          for model_name, model in models.items()
                                          for ratio in shuffle_ratio
                                          for train_idx, val_idx in kf.split(X)
                                         )

In [ ]:
directory = 'results/feature_shuffle/'
create_directory(directory)

results_shuffled_input = {
    "train": results_shuffled_data_train, 
    "val":results_shuffled_data_val, 
    "all": results_shuffled_data_all,
}

for k,result in results_shuffled_input.items():
    df_results = pd.DataFrame(result, columns=['Model', 'Noise level', 'AUC', 'Brier score', 'Prob true', 'Prob pred'])
    df_results.to_csv(directory+k+'.csv', index=False)
    df_results = process_df(df_results, n_folds)    
    df_results.to_csv(directory+k+'.csv', index=False)

### Subgroup analysis

Here we consider different types of subgroup analysis: both within MIMIC-III and across MIMIC-IV. <br> For each analysis we consider either stratifying on one of the included feature (e.g., age) in the model or on a external variable (e.g., gender, icu unit).  

In [ ]:
directory = 'results/subgroups/'
create_directory(directory)

#### MIMIC-III

**By gender**

In [ ]:
def subgroup_analysis(model_name, model, train_idx, val_idx, stratify_on='gender'):

    X_train, X_val = X.loc[train_idx], X.loc[val_idx]
    y_train, y_val = y.loc[train_idx], y.loc[val_idx]

    model.fit(X_train, y_train)
    
    stratified_perf = []
    
    for cat in mimic_3[stratify_on].dropna().unique():
        X_val_strat = X_val.loc[mimic_3[stratify_on]==cat]
        y_val_strat = y_val.loc[X_val_strat.index]
        y_pred = model.predict_proba(X_val_strat)[:,1]
        prob_true, prob_pred = calibration_curve(y_val_strat, y_pred)
        stratified_perf.append([model_name, stratify_on, cat, roc_auc_score(y_score=y_pred, y_true=y_val_strat), brier_score_loss(y_true=y_val_strat, y_proba=y_pred), prob_true, prob_pred])
        
    return stratified_perf

In [ ]:
results_subgroup_m3_gender = Parallel(n_jobs=-1, verbose=1)(delayed(subgroup_analysis)(model_name, model, train_idx, val_idx, variable) 
                                          for model_name, model in models.items()
                                          for variable in ['gender']
                                          for train_idx, val_idx in kf.split(X)
                                         )
results_subgroup_m3_gender = [x for xs in results_subgroup_m3_gender for x in xs]

In [ ]:
results_subgroup_m3_agegroup = Parallel(n_jobs=-1, verbose=1)(delayed(subgroup_analysis)(model_name, model, train_idx, val_idx, variable) 
                                          for model_name, model in models.items()
                                          for variable in ['age_group']
                                          for train_idx, val_idx in kf.split(X)
                                         )
results_subgroup_m3_agegroup = [x for xs in results_subgroup_m3_agegroup for x in xs]

In [ ]:
results_subgroup_m3_icu_unit = Parallel(n_jobs=-1, verbose=1)(delayed(subgroup_analysis)(model_name, model, train_idx, val_idx, variable) 
                                          for model_name, model in models.items()
                                          for variable in ['ICU_unit']
                                          for train_idx, val_idx in kf.split(X)
                                         )
results_subgroup_m3_icu_unit = [x for xs in results_subgroup_m3_icu_unit for x in xs]

In [ ]:
results_subgroup_analysis_m3 = {
    "m3_gender": results_subgroup_m3_gender, 
    "m3_age": results_subgroup_m3_agegroup, 
    "m3_icu_unit": results_subgroup_m3_icu_unit,
}

for k,result in results_subgroup_analysis_m3.items():
    df_results = pd.DataFrame(result, columns=['Model', 'Variable', 'Category', 'AUC', 'Brier score', 'Prob true', 'Prob pred'])
    df_results.to_csv(directory+k+'.csv', index=False)
    df_results = process_df(df_results, n_folds)    
    df_results.to_csv(directory+k+'.csv', index=False)

### Temporal validation

In [ ]:
def train_evaluate(model_name, model, X_tr, y_tr, df_test, stratify_on='gender'):

    model.fit(X_tr, y_tr)

    y_pred = model.predict_proba(df_test[X_tr.columns])[:,1]

    stratified_perf = []
    if stratify_on is not None:
        for cat in df_test[stratify_on].dropna().unique():
            X_eval_strat = df_test.loc[df_test[stratify_on]==cat][X_tr.columns]
            y_eval_strat = df_test.loc[df_test[stratify_on]==cat]['hospital_mortality']
            y_pred = model.predict_proba(X_eval_strat)[:,1]
            prob_true, prob_pred = calibration_curve(y_eval_strat, y_pred)
            stratified_perf.append([model_name, stratify_on, cat, roc_auc_score(y_score=y_pred, y_true=y_eval_strat), brier_score_loss(y_true=y_eval_strat, y_proba=y_pred), prob_true, prob_pred])
        
        return stratified_perf

    prob_true, prob_pred = calibration_curve(df_test['hospital_mortality'], y_pred)
    return model_name, roc_auc_score(y_score=y_pred, y_true=df_test['hospital_mortality']), prob_true, prob_pred    
    

In [ ]:
results_m4 = Parallel(n_jobs=-1, verbose=1)(delayed(train_evaluate)(model_name, model, X, y, mimic_4, stratify_on=None) 
                                          for model_name, model in models.items()
                                         )

In [ ]:
results_m4_gender = Parallel(n_jobs=-1, verbose=1)(delayed(train_evaluate)(model_name, model, X, y, mimic_4, stratify_on=variable) 
                                          for model_name, model in models.items()
                                          for variable in ['gender']
                                         )

results_m4_gender = [x for xs in results_m4_gender for x in xs]

In [ ]:
results_m4_age = Parallel(n_jobs=-1, verbose=1)(delayed(train_evaluate)(model_name, model, X, y, mimic_4, stratify_on=variable) 
                                          for model_name, model in models.items()
                                          for variable in ['age_group']
                                         )

results_m4_age = [x for xs in results_m4_age for x in xs]

In [ ]:
results_m4_icu_unit = Parallel(n_jobs=-1, verbose=1)(delayed(train_evaluate)(model_name, model, X, y, mimic_4, stratify_on=variable) 
                                          for model_name, model in models.items()
                                          for variable in ['ICU_unit']
                                         )

results_m4_icu_unit = [x for xs in results_m4_icu_unit for x in xs]

In [ ]:
results_m4_year = Parallel(n_jobs=-1, verbose=1)(delayed(train_evaluate)(model_name, model, X, y, mimic_4, stratify_on=variable) 
                                          for model_name, model in models.items()
                                          for variable in ['anchor_year_group']
                                         )

results_m4_year = [x for xs in results_m4_year for x in xs]

In [ ]:
directory = 'results/m4/'
create_directory(directory)

In [ ]:
results_m4_all = {
    "m4_gender": results_m4_gender,
    "m4_age": results_m4_age, 
    "m4_icu_unit": results_m4_icu_unit, 
    "m4_year": results_m4_year,
}

for k,result in results_m4_all.items():
    df_results = pd.DataFrame(result, columns=['Model', 'Variable', 'Category', 'AUC', 'Brier score', 'Prob true', 'Prob pred'])
    df_results.to_csv(directory+k+'.csv', index=False)
    df_results = process_df(df_results, n_folds)    
    df_results.to_csv(directory+k+'.csv', index=False)